In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
import statsmodels as sms
import scipy as cp
import math
import csv
import matplotlib.pyplot as plt
import openpyxl
#import outlier

# scikit-learn
# scikit-gof

%matplotlib
%matplotlib inline

########### Section 0 ###########
##### Pre-defined functions #####
#################################

###############################
# Data transformation functions
###############################

# 1. Function to take difference (PD, LD, FD)
# pdSeries =  a panda Series object or a list of A pre-transformed variable
# transformstr =  a string of PD or LD or FD
def transform(pdSeries, transformstr):
    if 'LD' in transformstr:
        posttrans = np.log(pdSeries).diff()      
    elif 'FD' in transformstr:
        posttrans = pdSeries.diff()
    elif 'PD' in transformstr:
        posttrans = np.exp(np.log(pdSeries).diff()) - 1
    elif 'None' in transformstr:
        posttrans = pdSeries
    return posttrans

# 2. Function to take lag
# pdSeries = a panda Series object or a list of A pre-transformed variable
# order = an integer of # of lags to take
def takelag(pdSeries, orderint):
    postlag = pd.Series(np.roll(pdSeries, orderint), index = pdSeries.index)
    for i in range(orderint):
        postlag[i] = None
    return postlag

# 3. Call above two fcns to IVs within a scenario
# scenariodf = a dataframe of one scenario IVs with pre-transformed data. Note: Columns = pre-transformed IV names
# transdic = a dictionary defining each variable and its corresponding difference transformations
#            for example {'pre-transV1name': 'PD', 'pre-transV2name': 'FD', 'pre-transV3name': 'LD'}
# lagdic = a dictionary defining each variable and its corresponding lag taken if any
#           for example {'pre-transV1name': 1, 'pre-transV2name': 2, 'pre-transV3name': 4}
# the function adds post-transformed variables with its newnames (pre-transname_trans_lagNum)to the scenariodf 
def aggtrans(scenariodf, transdic, lagdict):
    for i in scenariodf.columns:
        if i in transdic.keys():
            if i in lagdict.keys():
                laggedSeries = takelag(scenariodf[i], lagdict[i])
                scenariodf[i + '_' + transdic[i] + '_' + 'lag'+ str(lagdict[i])] = transform(laggedSeries, transdic[i])
            else:
                scenariodf[i + '_' + transdic[i]] = transform(scenariodf[i], transdic[i])
    return scenariodf

# 4. function to convert post-transformed y-hat to level y-hat
# yScenarioSeries = a pandas Series object of the predicted post-transformed y-hat of that scenario with date as index
# DVtransform = a string of y transformation (FD, LD or PD)
# jumpoff =  a float of the jumpoff value
# jumpoffdate = a datetime object or a string represent the jumpoffdate. 
### Note: jumpoffdate has the same format and type as yScenarioSeries index
def backtolevel(yScenarioSeries, DVtransform, jumpoff, jumpoffdate):
    yLevels = [None] * (len(yScenarioSeries) + 1)
    yLevels[0] = jumpoff
    if 'LD' in DVtransform:
        for i in range(1,len(yScenarioSeries)+1):
            yLevels[i] = yLevels[i-1] * np.exp(yScenarioSeries.iloc[i-1])
    if 'FD' in DVtransform:
        for i in range(1,len(yScenarioSeries)+1):
            yLevels[i] = yLevels[i-1] + yScenarioSeries.iloc[i-1]
    if 'PD' in DVtransform:
        for i in range(1,len(yScenarioSeries)+1):
            yLevels[i] = yLevels[i-1] * (1 + yScenarioSeries.iloc[i-1]) 
            
    yLevels = pd.Series(yLevels, index = yScenarioSeries.index.insert(0, jumpoffdate))        
    return yLevels

######################################################################
# function to combine all scenarios into one single dataframe to chart
######################################################################

# vaname =  a string of the variable name that you want to pull out and chart
# histdf = a dataframe of historical variables with 
#            variable names as column names and 
#            a 'Datetime' column of datetime obejct representing time
# jumpoffdate = a datetime object of the jump-off date
# scenariodflist = a list of dataframes of scenario data (one scenario, one DF):
#                  all DFs in this list have the same shape and same column names of variable names
#                  each DF has a 'Datetime' column of datetime obejct representing time
# index = a list of strings of scenario names:
#         order and length must match scenariodflist: scenariodflist = [dfA, dfB, dfC] then index = ['A','B','C']

def combineallscenarios(vname, histdf, jumpoffdate, scenariodflist, index):
    hist1 = histdf[histdf['Datetime'] <= jumpoffdate][vname]
    scenariodfinterim = [i[i['Datetime'] >= jumpoffdate][vname] for i in scenariodflist]
    scenariodfinterim.insert(0,hist1)
    variable = pd.DataFrame(scenariodfinterim, index)
    result = variable.transpose() 
    return result

######################################
# Dynamic in-sample back test function
######################################

# databtdf = a dataframe of all transformed IVs within the exact historical regression period with date as the index
# DVdf = a dataframe of historical DVs' levels (i.e. pre-transformation) with date as the index
# IVnamelist = a list of post-transformed IV names that used in this back-test
# DVstr = a string of pre-transformed DV name that used in this back-test
# forecast period = 9 for quarterly forecast or 27 for monthly forecast
# modelfit = the OLS model.fit() object
# transstr =  a string of the DV transformation. 'PD' as growth, 'LD' as log difference, 'FD' as first difference
# modelID = an integer of model ID to name the output csv file
### Note:databtdf and DVdf should have index with the same type or format
### Note: If you date index in dataframes are strings (e.g.'3/31/2009'), 
###          you need to sort result[2] in the csv file by date to get the diagnal in-sample forecast matrix
### Note:DVdf contains initial date value while databtdf may not if DV is differenced before modeling
### Note:This function called the pre-defined function backtolevel()
# The function returns [a TS of MAPEs, a TS of MPEs, dynamic in-sample, dynamic MAPEs, dynamic MPEs]

def dynamicbacktest(databtdf, DVdf, IVnamelist, forecastperiod, modelfit, transstr, DVstr, modelID):
    avgMAPE = []
    avgMPE = []    
    DVtoMerge = DVdf[DVstr]
    MAPEtoMerge = DVdf[DVstr]
    MPEtoMerge = DVdf[DVstr]
    for i in range(0, len(databtdf)):
        # define each regression period end point
        if i + forecastperiod - 1 <= len(databtdf):
            end_i = i + forecastperiod
        else:
            end_i = len(databtdf)
        
        # X and y-hat for each regression period
        X_bt = sm.add_constant(databtdf[IVnamelist])[i:end_i]  
        ytrans_bt = modelfit.predict(X_bt)                    

        # reset jumpoff value for each regression period
        thisjumpoff = DVdf[DVstr][list(DVdf[DVstr].index).index(ytrans_bt.index[0])-1]
        
        # reset jumpoff date for each regression period
        thisjumpoffdate = DVdf[DVstr].index[list(DVdf[DVstr].index).index(ytrans_bt.index[0])-1]
        # y levels for each regression period
        y_bt = pd.DataFrame(backtolevel(ytrans_bt, transstr, thisjumpoff, thisjumpoffdate), 
                        columns = [thisjumpoffdate])[1:]
        # calculate MAPE and MPEs
        MAPEdf = pd.merge(pd.DataFrame(DVdf[DVstr]), y_bt, how = 'inner', right_index = True, left_index = True)
        MAPEdf['MAPE'] = abs(MAPEdf[DVstr] - MAPEdf[thisjumpoffdate]) / MAPEdf[DVstr]
        thisMAPE = np.mean(MAPEdf['MAPE'])
        avgMAPE.append(thisMAPE)
        MAPEdf['MPE'] = (MAPEdf[thisjumpoffdate] - MAPEdf[DVstr]) / MAPEdf[DVstr]
        thisMPE = np.mean(MAPEdf['MPE'])
        avgMPE.append(thisMPE)
    
        # record each 9Q MAPEs and MPEs
        MAPEtoMerge = pd.merge(pd.DataFrame(MAPEtoMerge), pd.DataFrame(MAPEdf['MAPE']), how = 'outer', right_index = True, left_index = True)
        MPEtoMerge = pd.merge(pd.DataFrame(MPEtoMerge), pd.DataFrame(MAPEdf['MPE']), how = 'outer', right_index = True, left_index = True)
        
        thiselement = pd.DataFrame([thisjumpoff], index = [thisjumpoffdate], columns = [thisjumpoffdate])
        y_bt_with_jumpoff = pd.concat([thiselement, y_bt])
        #record each 9Q predictions including the jumpoff value of this period
        DVtoMerge = pd.merge(pd.DataFrame(DVtoMerge), y_bt_with_jumpoff, how = 'outer', right_index = True, left_index = True)
    
    # average MAPE series to chart
    MAPEchart = pd.DataFrame([avgMAPE, avgMPE], index = ['avgMAPE', 'avgMPE']).transpose().set_index(databtdf.index)
    MAPEchart['threshold'] = 0.2
    # final average MAPEs cross all historical 9Q period
    avgMAPE9Q = np.mean(avgMAPE[:(len(avgMAPE)-forecastperiod+2)])
    # final average MAPEs cross entire history including those less-than-9Q periods at the end
    avgMAPEfull = np.mean(avgMAPE)
    # average MPE series to chart
    MAPEchart['full period average'] = avgMAPEfull
    # final average MPEs cross all historical 9Q period
    avgMPE9Q = np.mean(avgMPE[:(len(avgMPE) - forecastperiod + 2)])
    # final average MPEs cross entire history including those less-than-9Q periods at the end
    avgMPEfull = np.mean(avgMPE)
    # record above outputs into a dataframe for output purpose
    MAPEoutput = pd.DataFrame([avgMAPE9Q, avgMAPEfull, avgMPE9Q, avgMPEfull], 
                         index = ['avg.MAPE.9Q', 'avg.MAPE.full', 'avg.MPE.9Q', 'avg.MPE.full'], columns = ['Value'])
        
    backtestresults = [MAPEchart, MAPEoutput, DVtoMerge, MAPEtoMerge, MPEtoMerge]
    sheetnames = ['avg MAPE series', 'final MAPE', 'Dynamic In-sample', 'Dynamic MAPEs', 'Dynamic MPEs']
    
    with pd.ExcelWriter(str(modelID)+'_out5_'+DVstr+'_DynamicBackTest.xlsx') as writer:
        for i in range(len(backtestresults)):
            backtestresults[i].to_excel(writer, sheet_name = sheetnames[i])
    
    return backtestresults


############################
# coefficient stability test
############################

# datareg =  A dataframe contains all post-transformed IVs and DVs of the exact regression period
# n = initial number of observations to include in the first-period of regression test
# DVname =  a string of post-transformed DV name
# IVname = a list of strings of post-transformed IV names

def stabilitytest(datareg, n, DVname, IVname):
    import statsmodels.api as sm
    pvalues = []
    coeff = []
    index = datareg.index[n-1:]
    y = datareg[DVname]
    X = sm.add_constant(datareg[IVname])
    
    for i in range(n-2, len(X)):
        thisX = X[:i+1]
        thisy = y[:i+1]
        thismodel = sm.OLS(thisy, thisX)
        thisres = thismodel.fit()
        coeff.append(thisres.params[1:])
        pvalues.append(thisres.pvalues[1:])
        stabilitycoeffdf = pd.DataFrame(coeff)
        stabilitypvaluedf = pd.DataFrame(pvalues)
        
    return [stabilitycoeffdf, stabilitypvaluedf]


#########################
## Updated Sensitivity ##
#########################

# function to shock by IV std
def sensIVstd(sensIVs, IVstds, IVstdratio, dvtrans, DVtoChart, jumpoff, jumpoffdate, shock, scenarios):
    y = []
    for i in range(0, len(sensIVs)):
        ythisscenario = []
        for j in IVname:
            thisX = sensIVs[i].copy()
            if shock == 'plusIVstd':          
                thisX[j] += IVstds[j]
            if shock == 'minusIVstd':
                thisX[j] -= IVstds[j]
            if shock == 'plusIVstdratio':
                thisX[j] *= (1+IVstdratio[j])
            if shock == 'muniusIVstdratio':
                thisX[j] *= (1-IVstdratio[j])
            thisyhat = res.predict(sm.add_constant(thisX))   
            thisylevel = backtolevel(thisyhat,dvtrans, jumpoff, jumpoffdate)
            ythisscenario.append(thisylevel)
        ythisscenariodf = pd.DataFrame(ythisscenario).transpose()
        ythisscenariodf.columns = IVname
        ythisscenariodf['Scenario'] = scenarios[i+1]
        ythisscenariodf['Shock'] = shock
        y.append(ythisscenariodf)
    return y

# function to shock by coeff std
def senscoeffstd(sensIVs, res,  dvtrans, DVtoChart, jumpoff, jumpoffdate, shock, scenarios):
    coeffs = res.params
    stds = res.bse
    y = []
    for i in range(0, len(sensIVs)):
        ythisscenario = []
        for j in IVname:
            thisX = sensIVs[i].copy()
            thiscoeff = coeffs.copy()
            if shock == 'pluscoeffstd':
                thiscoeff[j] += stds[j]
            if shock == 'minuscoeffstd':
                thiscoeff[j] -= stds[j] 
            thisyhat = sm.add_constant(thisX).dot(thiscoeff)  
            thisylevel = backtolevel(thisyhat, dvtrans, jumpoff, jumpoffdate)
            ythisscenario.append(thisylevel)
        ythisscenariodf = pd.DataFrame(ythisscenario).transpose()
        ythisscenariodf.columns = IVname
        ythisscenariodf['Scenario'] = scenarios[i+1]        
        ythisscenariodf['Shock'] = shock
        y.append(ythisscenariodf)
    return y

# function to shock IV by %
def sensIV(sensIVs, pct, dvtrans, DVtoChart, jumpoff, jumpoffdate, shock, scenarios):
    y = []
    for i in range(0, len(sensIVs)):
        ythisscenario = []
        for j in IVname:
            thisX = sensIVs[i].copy()
            if shock == 'plusIVpct':
                thisX[j] *= (1+pct)
            if shock == 'minusIVpct':
                thisX[j] *= (1-pct)
            thisyhat = res.predict(sm.add_constant(thisX))   
            thisylevel = backtolevel(thisyhat,dvtrans, jumpoff, jumpoffdate)
            ythisscenario.append(thisylevel)
        ythisscenariodf = pd.DataFrame(ythisscenario).transpose()
        ythisscenariodf.columns = IVname
        ythisscenariodf['Scenario'] = scenarios[i+1]
        ythisscenariodf['Shock'] = shock
        y.append(ythisscenariodf)
    return y

# calculate sensitivity results
def sensresults(ylevelforecastdflist, DVtoChart, IVname, y, forecasttype, scenarios):
    if forecasttype == 'Revenue':
        forecast = pd.DataFrame([np.sum(i[DVtoChart][1:]) for i in ylevelforecastdflist], index = scenarios[1:])
        forecast_shocked = pd.DataFrame([np.sum(i[IVname][1:]) for i in y], index = scenarios[1:])
    else:
        forecast = pd.DataFrame([np.mean(i[DVtoChart][1:]) for i in ylevelforecastdflist], index = scenarios[1:])
        forecast_shocked = pd.DataFrame([np.mean(i[IVname][1:]) for i in y], index = scenarios[1:])
    forecast.columns = ['Forecast']
    resultdf = pd.merge(forecast, forecast_shocked, how = 'outer', left_index = True, right_index = True)
    ychange = []
    for i in IVname:
        ychange.append(resultdf[i]/resultdf['Forecast'] - 1)
        resultdf['yChange by shocking_' + i] = (resultdf[i]/resultdf['Forecast'] - 1)
    return resultdf


### Other functions ####

# Data slicing function
def sliceforecsatperiod(scenariotransdf):
    df = scenariotransdf[scenariotransdf.columns[list(
        scenariotransdf.columns).index('Datetime'):]][scenariotransdf.Datetime >= forecaststart]
    df.set_index('Datetime', inplace = True)
    return df

########### Section 1 ###########
####### Data Preparation ########
#################################

####################################################
## 1.1 all user input parameters for this section ##
####################################################

# model ID as string
modelID = 19972

# specify csv file name
IVfilename = '19972IVretrieved.csv'
DVfilename = '19972DV.csv'

# for each variable retrived, specify its corresponding transformation used
# use 'LD' for log difference
# use 'FD' for first difference
# use 'PD' for growth

IVDVtrans = {'Ratio': 'LD', 'TotalComp': 'LD',
             'USRCDS5YE':'LD', 'USCPIA':'LD', 'USBIXBOACCCRE':'LD',
             'USOICSENTA':'LD', 'USVOLNASDAQT':'LD'}

# for each variable that takes lag, specify in the dictionairy below
IVDVlag = {'USOICSENTA': 1, 'USCPIA':1}

# specify level startdate, regression startdate, forecast startdate and jumpoff date.
# use 1st date by default for each month end. i.e. 6/1/2020 means the end of Jun in 2020
yLevelstart = dt.date(2008, 3, 1)
regstart = dt.date(2008, 6, 1)
regend = dt.date(2020, 12, 1)
forecaststart = dt.date(2021, 3, 1)
jumpoffdate = dt.date(2020, 12, 1)


# specify scenario names in the order of scenarios retrieved with hist at the beginning
# scenario order needs to the same as the order you process the forecast
scenarios = ['Hist', 'FRB Base','FRB Sev', 'IHC Base', 'IHC Idio','IHC Sev']
scenarioslableinIVretrievefile = ['Historical','FRBBaseline', 
                                  'FRBSeverely', 'IHCBaseline', 
                                  'BankingCris', 'SevereBanki']

####################################################
## 1.2 prepare data using pre-defined functions ####
####################################################
# read csv from IV retrieved and set date as dataframe index
IVs = pd.read_csv(IVfilename)
IVs.set_index('Date', inplace = True)
IVdate = [dt.datetime.strptime(i, '%Y-%m-%d') for i in IVs.index]
IVs['Datetime'] = [dt.date(i.year, i.month, 1) for i in IVdate]
#del IVs['Unnamed: 0']

# slice IV data by scenarios. Note the same scenario order as scenarios list defined above
hist = IVs[IVs['Scenario'] == scenarioslableinIVretrievefile[0]]
IVscenariodflist = [pd.concat([hist, IVs[IVs['Scenario'] == scenarioslableinIVretrievefile[i]]]) 
                    for i in range(1, len(scenarioslableinIVretrievefile))]

# read DV from csv file and set date as dataframe index
DV = pd.read_csv(DVfilename)
DV.set_index('Date', inplace = True)

DVdate = [dt.datetime.strptime(i, '%Y/%m/%d') for i in DV.index]
DV['Datetime'] = [dt.date(i.year, i.month, 1) for i in DVdate]

DV.head()

# transform IVs using pre-defined functions
IVscenariotransdflist = [aggtrans(df, IVDVtrans, IVDVlag) for df in IVscenariodflist]
# get post-transformation IVs only (i.e. those on the right of 'Datetime' column) for regression period only
IVreg = IVscenariotransdflist[0][IVscenariotransdflist[0].columns[
                                 list(IVscenariotransdflist[0].columns).index('Datetime'):]
                                ][IVscenariotransdflist[0].Datetime >= regstart]
IVreg.set_index('Datetime', inplace = True)

# transform DVs using pre-defined functions
DVtrans = aggtrans(DV, IVDVtrans, IVDVlag)
# get post-transformation DVs only (i.e. those on the right of 'Datetime' column) for regression period only
DVreg = DVtrans[DVtrans.columns[list(DVtrans.columns).index('Datetime'):]][DVtrans.Datetime >= regstart]                                                                       
DVreg.set_index('Datetime', inplace = True)
DVreg = DVreg[DVreg.index <= regend]

# final dataset of both post-transformed IVs and DV for regression
datareg = pd.merge(DVreg, IVreg, how = 'left', left_index = True, right_index = True)
transVnames = list(datareg.columns)

# final IV datasets for forecast
IVscenariotransdflist_forecastonly = [sliceforecsatperiod(i) for i in IVscenariotransdflist]

# output all transformed data
with pd.ExcelWriter(str(modelID)+ '_challenger_out1_'+'All_IV_DV_postTrans.xlsx') as writer:
    datareg.to_excel(writer, sheet_name = 'regression hist')
    for i in range(len(IVscenariotransdflist_forecastonly)):
        IVscenariotransdflist_forecastonly[i].to_excel(writer, sheet_name = scenarios[i+1])

# get Variable names to be displayed for next steps
IVnames = list(IVs.columns)
IVnames.remove('Scenario')
IVnames.remove('Datetime')

DVnames = list(DV.columns)
DVnames.remove('Datetime')

print('post-transformation variable names:', transVnames)
print('pre-transformation IV names: ', IVnames)
print('pre- and post- transformation DV names: ', DVnames)

########### Section 2 ############
#### Input data visualization ####
##################################

####################################################
## 2.1 all user input parameters for this section ##
####################################################

# list the IV and DV names that you want to chart out
# can copy paste the output from last section

# model ID defined in Section 1.1
# modelID

IVtoChart = ['USRCDS5YE', 'USCPIA', 'USBIXBOACCCRE', 'USOICSENTA', 'USVOLNASDAQT']
DVtoChart = 'TotalComp'
##########################
# 2.2 Chart below ########
##########################

# Chart DV
#DV[DVtoChart].plot(figsize=(10,5), title = DVtoChart);

# Chart each IV using the pre-defined fcn
#for i in IVtoChart:
#    combineallscenarios(i, IVs, jumpoffdate, IVscenariodflist, scenarios).plot(figsize=(10,5), 
#                                                              style = ['k-', 'g-', 'r-', 'g--', 'y--','r--'], title = i);

# output all historical data
allIVs = [combineallscenarios(i, IVs, jumpoffdate, IVscenariodflist, scenarios) for i in IVtoChart];
with pd.ExcelWriter(str(modelID)+'_challenger_out2_'+'All_IV_preTrans.xlsx') as writer:
    for i in range(len(allIVs)):
        allIVs[i].to_excel(writer, sheet_name = IVtoChart[i])    

########### Section 3 ###########
## regression model and tests ###
#################################

####################################################
## 3.1 all user input parameters for this section ##
####################################################

# variables defined in section 2.1
DVname = 'TotalComp_LD'
IVname = ['USBIXBOACCCRE_LD','USRCDS5YE_LD']

# If you did not use Section 1 and 2 above but start using the code from Section 3, 
# plesae make sure your input file with the same format as the file below, 
# 999_out1_RegIVDV_postTran_Hist.csv

# Read your input file as pd.DataFrame datareg by uncommenting the two lines below
# dataregfilename = 'filename.csv'
# datareg = pd.read_csv('dataregfilename')

##############################
##### 3.2 OLS model #########
##############################
OLStestresult = []

import statsmodels.api as sm

y = datareg[DVname]
X = sm.add_constant(datareg[IVname])
model = sm.OLS(y, X)
res = model.fit()
resid = res.resid

coeffdf = pd.DataFrame([res.params, res.bse, res.pvalues]).transpose().rename(columns = {0:'coefficient', 
                                                                                         1:'std err', 
                                                                                         2:'pvalue'})
reginfodf = pd.DataFrame([res.rsquared_adj, res.f_pvalue, res.df_model, res.nobs], 
             index = ['Adj-Rsquared', 'F-test Pvalue','DF model', 'observations'])

OLStestresult.append(coeffdf)
OLStestresult.append(reginfodf)

##############################
### 3.3 OLS Model Test #######
##############################

##### Stationarity test #####
from statsmodels.tsa.stattools import adfuller

# this function should give same results as stationarity.test in R from aTSA package
# this function returns a tuple of adf_statistic, p-value, #lags used, #observations used, critical values and best IC

datareg['residual'] = resid
Vtotest = IVname.copy()
Vtotest.append(DVname)
Vtotest.append('residual')

stat = []
pvalue = []
lag = []
Type = []
name = []
result =[]

for v in Vtotest:
    for r in ['nc', 'c']:
        for i in range(0,4):
            x = adfuller(datareg[v], maxlag=i+1, regression = r, autolag=None)
            stat.append(x[0])
            pvalue.append(x[1])
            lag.append(x[2])
            Type.append(r)
            name.append(v)
            if x[1] < 0.01:
                result.append('High pass')
            elif x[1] < 0.05:
                result.append('Medium pass')
            elif x[1] < 0.1:
                result.append('Low pass')
            else:
                result.append('Fail')
        
staresults = pd.DataFrame([name, Type, stat, pvalue, lag, result]).transpose()
staresults.rename(columns = {0:'Stationarity', 1:'Type', 2:'Stat', 
                             3:'P-value', 4:'Lag', 5:'Result'}, inplace = True)      

OLStestresult.append(staresults)

##### VIF test for Multicollinearity ######
from statsmodels.stats.outliers_influence import variance_inflation_factor

# This function should give the same results as vif in R from car package
vif = pd.DataFrame([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index=X.columns)
vif.rename(columns = {0:'VIF'}, inplace = True)

OLStestresult.append(vif)

##### BG Test autocorrelation #####
bgresults = pd.DataFrame(index = ['statistic', 'p-value'])

from statsmodels.stats.diagnostic import acorr_breusch_godfrey
# This function should give the same results chi-square test as bgtest in R from lmtest package
# This function returns a tuple of chi-statistic, chi-test p-value, F-statistic, F-test p-value

for i in range(1,5):
    bgresults[i] = pd.Series(acorr_breusch_godfrey(res, nlags = i)[0:2], index = bgresults.index)

bgresults = bgresults.transpose()
bgresults.index.rename('BG test order', inplace = True)

OLStestresult.append(bgresults)

##### BP test for Homoscedasticity #####
from statsmodels.stats.diagnostic import het_breuschpagan

# This function should give the same results chi-square test as bptest in R from lmtest package
# This function returns a tuple of chi-statistic, chi-test p-value, F-statistic, F-test p-value
bpresult = pd.DataFrame(list(het_breuschpagan(resid, X)[0:2]), index = ['Stat', 'p-value'])
bpresult.rename(columns = {0:'BP Test'}, inplace = True)

OLStestresult.append(bpresult)

##### Ramsey test for linearty #####
#from statsmodels.stats.diagnostic import linear_reset

# This function should give the same results as resettest in R from lmtest package when use_f = True
# here power = 3 should correspond to power = 2:3 in R
#stat = []
#pvalue = []
#Type = []
#Power = []

#for t in ['fitted', 'exog']:
#    for power in [2,3,4]:
#        x = linear_reset(res, power = power, test_type = t, use_f = True).summary()
        #print(x)
#        stat.append(float(x.split(',')[0].split('[')[2].split(']')[0]))
#        pvalue.append(float(x.split(',')[1].split('=')[1]))
#        Type.append(t)
#        Power.append(power)

#ramseyresult = pd.DataFrame([Type, Power, stat, pvalue]).transpose()
#ramseyresult.rename(columns = {0:'Ramsey test type', 1:'Power', 2:'Stat', 3:'p-value'}, inplace = True)

#OLStestresult.append(ramseyresult)

##### Normality test #####
normresult = pd.DataFrame(index = ['Stat', 'p-value / 0.05 critical value for AD'])

# KS Test
from statsmodels.stats.diagnostic import lilliefors
# This function should give the same results as lillie.test in R from nortest package
normresult['KS'] = pd.Series(lilliefors(resid, 'norm'), index = normresult.index)

# JB Test
from scipy.stats import jarque_bera
# This function should give the same results as jarque.bera.test in R
normresult['JB'] = pd.Series(jarque_bera(resid), index = normresult.index)

# Shapiro Test
from scipy.stats import shapiro
# This function should give the same results as shapiro.test in R
normresult['Shapiro'] = pd.Series(shapiro(resid), index = normresult.index)

# AD test
#from skgof import ks_test, cvm_test, ad_test
# This function should give the same results as ad.test in R
#adtest = cp.stats.anderson(resid, 'norm')
#normresult['AD'] = pd.Series([adtest[0],adtest[1][2]], index = normresult.index)

# cvm Test
# This function DOES NOT give the same results as cvm.test in R
# normresult['CVM'] = pd.Series(cvm_test(resid, 'norm'), index = normresult.index)
normresult = normresult.transpose()

OLStestresult.append(normresult)

#with open(str(modelID)+'_out3_'+ DVname + '_' + IVname[2] +'_OLSwithTest.csv', 'w') as f:
#    for i in range(0,len(OLStestresult)):
#        OLStestresult[i].to_csv(f)     

########### Section 4 ###########
########### Forecast  ###########
#################################

####################################################
## 4.1 all user input parameters for this section ##
####################################################

# variable defined in Section 1.1
# modelID
# IVDVtrans
# jumpoffdate

# variable 2.1
# DVtoChart

# variable 3.1
# DVname
# IVname

#############################
## 4.2 predicting  ##
#############################

# remember transformed IV of forecast period generated in Section 1.2
# pd DataFrame of BCf, CVEf, CVSf and IHCBf in IVscenariotransdflist_forecastonly

# post-transformation prediction
yposttransforecastpdlist = [res.predict(i) for i in 
                            [sm.add_constant(j[IVname]) for j in 
                             IVscenariotransdflist_forecastonly]]

# find jumpoff of y or define it yourself if needed
# jumpoff = DV[DVtoChart].tail(1).values[0]
jumpoff = DV[DVtoChart][DV['Datetime'] == jumpoffdate].values[0]

# use pre-dedined fcn to transfer predicted y back to levels for all scenarios
ylevelforecastdflist = [pd.DataFrame(backtolevel(i,IVDVtrans[DVtoChart], jumpoff, jumpoffdate)) 
                                               for i in yposttransforecastpdlist]
# prepare data set for the next pre-defined fcn 
DV.set_index('Datetime', inplace = True)
DV['Datetime'] = DV.index
for i in ylevelforecastdflist:
    i['Datetime'] = ylevelforecastdflist[0].index
    i.rename(columns = {0:DVtoChart}, inplace = True)
    
cp.stats.anderson(resid, 'norm')    
    
# use pre-defined function to construct y-data frame of all scenarios
yallscenariosdf = combineallscenarios(DVtoChart, DV, jumpoffdate, ylevelforecastdflist, scenarios)

# output forecast
yallscenariosdf.to_excel(str(modelID)+'_challenger_out4_'+ DVname + '_forecast.xlsx')

# Chart y with forecast
yallscenariosdf.plot(figsize=(10,5), style = ['k-', 'g-', 'r-', 'g--', 'y--','r--'], title = 'Ratio from credit spread');

########### Section 5 ###########
###### Dynamic  Backtest  #######
#################################

####################################################
## 5.1 all user input parameters for this section ##
####################################################

# User inputs defined in Section 1.1
# modelID
# yLevelstart
# regstart
# jumpoffdate
# IVDVtrans

# variable 2.1
# DVtoChart

# variable 3.1
# DVname
# IVname

# define forecast period 9 for quarterly, 27 for monthly
forecastperiod = 9

####################
## 5.2 back-test ###
####################

# All parameters used in the function were defined in previous sections except for databtdf and DVdf
# define databtdf and DVdf
databt = datareg[regstart <= datareg.index][datareg.index <= jumpoffdate] 
DV.set_index('Datetime', inplace = True)
DV['Datetime'] = DV.index

# Call the pre-defined backtestresults function
backtestresults = dynamicbacktest(databt, DV, IVname, 9, res, IVDVtrans[DVtoChart], DVtoChart, modelID)

# Output chart and MAPEs
backtestresults[0].plot(figsize=(10,5), style = ['g-', 'b-', 'r--', 'y-'], title = DVtoChart + ' Back Test MAPEs');
backtestresults[2].plot(legend = False, figsize=(10,5), style = ['k^-'], title = DVtoChart +' Dynamic Back Test');
print(backtestresults[1])

################## Section 6 ####################
#### coefficient stability and OOT back test ####
#################################################

####################################################
## 6.1 all user input parameters for this section ##
####################################################

# variables defined in 1.1
# modelID
# yLevelstart
# regstart
# jumpoffdate
# IVDVtrans

# variable 2.1
# DVtoChart

# variable 3.1
# DVname
# IVname

# variable defined 5.1
# forecastperiod

# define minimal number of obs for the first testing period
n = 30

##############################
# 6.2 Dynamic Stability Test #
##############################

# Call the pre-defined function of stabilitytest
stabilityresult = stabilitytest(datareg, n, DVname, IVname)

stbltcoeffdf = stabilityresult[0].set_index(datareg.index[n-2:])
stbltpvaluedf = stabilityresult[1].set_index(datareg.index[n-2:])

# output to excel
with pd.ExcelWriter(str(modelID)+'_ccar2020_out6_'+DVname+'_StabilityTest.xlsx') as writer:
    stbltcoeffdf.to_excel(writer, sheet_name = 'coefficient stability')
    stbltpvaluedf.to_excel(writer, sheet_name = 'pvalue stability') 

stbltcoeffdf.plot(figsize=(10,5), style = ['g-', 'b-', 'k-', 'y-'], title = str(modelID)+ DVname +' Coefficient Stability');
stbltpvaluedf.plot(figsize=(10,5), style = ['g-', 'b-', 'k-', 'y-'], title = str(modelID)+ DVname +' P-value Stability');

################## Section 7 ####################
################ OOT backtest ###################
#################################################

####################################################
## 7.1 all user input parameters for this section ##
####################################################

# variables defined in 1.1
# modelID
# yLevelstart
# regstart
# jumpoffdate
# IVDVtrans

# variable 2.1
# DVtoChart

# variable 3.1
# DVname
# IVname

# variable defined 5.1
# forecastperiod

dataregOOT = datareg[:len(datareg) - forecastperiod]
yOOT = dataregOOT[DVname]
XOOT = sm.add_constant(dataregOOT[IVname])
modelOOT = sm.OLS(yOOT, XOOT)
resOOT = modelOOT.fit()

datapredOOT = datareg[- forecastperiod:]
OOTjumpoffdate = dataregOOT.tail(1).index.values[0]

OOT = pd.merge(OOTy, pd.DataFrame(DV[DVtoChart]), how = 'inner', left_index = True, right_index = True)

OOT['MAPE'] = abs(OOT['OOT forecast'] - OOT[DVtoChart])/OOT[DVtoChart]
OOTMAPE = np.mean(OOT['MAPE'][1:])

print(OOT)
print('OOT average MAPE is:')
print(OOTMAPE)
OOT[['OOT forecast', DVtoChart]].plot(figsize=(10,5), style = ['g-', 'b-', 'r--', 'y-'], 
                                      title = DVtoChart + ' OOT Back Test');